#  Algoritmo de modelación de tópicos de documentos digitales

### Autores

#### Luis Roberto Polo Bautista
#### Karen Vanessa Martínez Acevedo 

#### El objetivo del algoritmo es asignar áreas temáticasa de documentos digitales en PDF, que sirva como herramienta de apoyo al análisis temático dentro de la organización de la información,  con el fin de ser implementado en el desarrollo de vocabularios controlados, como: taxonomías, tesauros, ontologías, etc.

### Conversióndel texto completo de documentos

#### En esta parte se utilizó el módulo PyMuPDF de Python, que entre otras funciones permite la identificación y conversión del texto de diferentes tipos de documentos a archivos editables. Es importante señalar que, el funcionamiento de este algoritmo toma como base documentos en  formato  PDF,  con  el  módulo PyMuPDF se  hace  uso  del  Reconocimiento  Óptico  de Caracteres,  que  permita  generar  archivos de texto  simple  (archivos  con  extensión  txt)  que contengan el texto completo de los documentos.

In [ ]:
# importar los módulos necesario
import fitz

In [ ]:
# Importar el documentoen formato PDF
documento_pdf="c:/"#<<<<<<<<-----Escribe la ruta donde se encuentra el documento PDF----->>>>>>>>
# Leer el documento mediante el módulo PyMuPDF
documento=fitz.open(documento_pdf)
# Mostrar el número de páginas y los datos bibliográficos
print ("Número de páginas del documento: ",documento.pageCount)
print ("Datos bibligráficos: ",documento.metadata)

In [ ]:
# Declarar un variable que contenga el nombre del nuevo archivo en formato txt
documento_txt=open(documento_pdf + ".txt", "wb")
 
for paginas in documento: # Para todas las páginas del documento  hacer: 
    texto=paginas.getText().encode("utf8") # Conversión del texto en formato UTF-8
    documento_txt.write(texto) # Escribir el texto en un archivo con la extensión definida
    documento_txt.write(b"\n-----\n")# Definir dentro del archivo las divisiones de cada página
documento_txt.close() #Cerrar el documento

### Identificación de temas relevantes y modelación visual

#### En  la  sección  de  identificación  de  temas se  utilizaron  los  módulos Numpy  y Scikit-Learn para el cálculo de frecuencia y vectores de palabrasy el modelo de Asignación Latente de Dirichlet (ALD) para la identificación de los temas relevantes. Para la visualización dinámica de los temaspor medio de HTML se utilizóel módulo pyLDAvis.

In [ ]:
# Importar los módulos necesarios
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
from sklearn.decomposition import LatentDirichletAllocation as LDA
import os
from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer 

os.chdir('..')

In [ ]:
# Importar el archivo de texto simple con el preprocesamiento realizado 
with open ('c:/Users/Luis/Desktop/book2.pdf.txt', 'r', encoding="utf-8") as archivo: #<<<<<<<<-----Escribe la ruta donde se encuentra el documento txt----->>>>>>>>
    texto= archivo.read()

In [ ]:
# Eliminar los artículos, pronombres, preposiciones, etc., del texto 
agregar = ['No', 'text', 'returned', 'unable', 'to', 'reach', 'website.', 'one', 'new', '--', 'us'] #<<<<<<<<-----Escribe las palabras que no quiere que sean consideradas a parte de las palabras vacías----->>>>>>>>
stop_words= (stopwords.words('spanish')) # <<<<<<<<-----para un texto en inglés, reemplazar 'spanish' por 'english' ----->>>>>>>>
stop_words.extend (agregar)
word_tokens= word_tokenize(texto.lower())
word_tokens= list(filter(lambda token: token not in string.punctuation, word_tokens))

In [ ]:
bolsa_palabras=[]
for palabra in word_tokens: 
    if palabra not in stop_words:
        filtro.append(palabra)

In [ ]:
# normalizar todas las formas de una misma palabra 
lemmatizer = WordNetLemmatizer()
lemmatized_output = [lemmatizer.lemmatize(w) for w in bolsa_palabras]

In [ ]:
# Calcular los vectores de las palabras individuales que conforman cada frase 
vectores_palabras = CountVectorizer()
transformar_vectores = vectores_palabras.fit_transform(lemmatized_output)

In [ ]:
# Crear una función que extraiga los vectores de las palabras y los guarde en una variable
def mostrar_tópicos(model, vectores_palabras, n_top_words):
    palabras = vectores_palabras.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([palabras[i]
                        for i in topic.argsort()[:-n_top_words - 1:- 1]]))

In [ ]:
# Declarar en una variable el número de temas a procesas
número_tópicos = 4 #<<<<<<<<-----Escribe el número de temas a mostrar----->>>>>>>>
# Declarar en una variable el número de palabras relacionadas a los temas
número_palabras = 30 #<<<<<<<<-----Escribe el número de palabras a considerar----->>>>>>>>

In [ ]:
# Procesar los datos ingresados con base a modulo ALD
lda = LDA(n_components=número_tópicos, n_jobs=-1)
lda.fit(transformar_vectores)

In [ ]:
# Mostrar los datos procesados con base al vector asignado a cada palabra, así como el número de palabras relacionados a los tópicos 
print("Tópicos encontrados mediante ALD:")
mostrar_tópicos(lda, vectores_palabras, número_palabras)

In [ ]:
# Declarar en una variable el nombre del archivo que mostrara los temas de forma dinámica 
LDAvis_data_filepath = os.path.join('./tópic_'+str(número_tópicos))#<<<<<<<<-----Escribe el nombre del archivo----->>>>>>>>
# Procesar los datos para que se puedan mostrar de forma dinámica
if 1 == 1:LDAvis_prepared = sklearn_lda.prepare(lda, transformar_vectores, vectores_palabras)

In [ ]:
# Preparar el archivo con los datos a mostrar
with open(LDAvis_data_filepath, 'wb') as f:
    pickle.dump(LDAvis_prepared, f)
        
with open(LDAvis_data_filepath, 'rb',) as f:
    LDAvis_prepared = pickle.load(f)

In [ ]:
# Guardar el archivo en una ruta determinada con extensión HTML

pyLDAvis.save_html(LDAvis_prepared, 'c:/'+ str(LDAvis_data_filepath) +'.html')#<<<<<<<<-----Escribe la ruta donde se quiere guardar el archivo HTML---->>>>>>>>